In [1]:
# Install your library here, for example the fynesse template
# is set up to be pip installable
%pip install git+https://github.com/jeffrey-22/ads.git
import fynesse
# 2 min

  Cloning https://github.com/jeffrey-22/ads.git to c:\users\86189\appdata\local\temp\pip-req-build-1b_apck0
  Resolved https://github.com/jeffrey-22/ads.git to commit e21089a9ff6bf55e4f41f5450924c78bb4ea4402
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/jeffrey-22/ads.git 'C:\Users\86189\AppData\Local\Temp\pip-req-build-1b_apck0'


In [141]:
pp_database_details = fynesse.access.retreive_database_details()
# fynesse.access.create_database(pp_database_details)
pp_database_conn = fynesse.access.create_connection(pp_database_details)

In the box below, briefly describe what the schema is doing.

```GIVE YOUR WRITTEN ANSWER HERE```

In [46]:
# Write the code you need for creating the table, downloading and uploading the data here. You can use as many code blocks as you need.
import os
os.makedirs("tmp_data", exist_ok=True)
downloaded_pathnames = fynesse.access.download_price_data()
print(f"Downloaded files: {downloaded_pathnames}")
# 20 min

Downloaded files: {'tmp_data\\pp-2021-part2.csv', 'tmp_data\\pp-2014-part2.csv', 'tmp_data\\pp-1997-part1.csv', 'tmp_data\\pp-2010-part2.csv', 'tmp_data\\pp-2019-part2.csv', 'tmp_data\\pp-2021-part1.csv', 'tmp_data\\pp-2003-part2.csv', 'tmp_data\\pp-2020-part1.csv', 'tmp_data\\pp-2012-part1.csv', 'tmp_data\\pp-2022.csv', 'tmp_data\\pp-2011-part2.csv', 'tmp_data\\pp-2006-part2.csv', 'tmp_data\\pp-2018-part1.csv', 'tmp_data\\pp-2007-part2.csv', 'tmp_data\\pp-2005-part1.csv', 'tmp_data\\pp-1998-part2.csv', 'tmp_data\\pp-2013-part1.csv', 'tmp_data\\pp-2013-part2.csv', 'tmp_data\\pp-2009-part2.csv', 'tmp_data\\pp-2007-part1.csv', 'tmp_data\\pp-2006-part1.csv', 'tmp_data\\pp-2000-part2.csv', 'tmp_data\\pp-1997-part2.csv', 'tmp_data\\pp-1995-part2.csv', 'tmp_data\\pp-2005-part2.csv', 'tmp_data\\pp-2014-part1.csv', 'tmp_data\\pp-1998-part1.csv', 'tmp_data\\pp-2016-part1.csv', 'tmp_data\\pp-2016-part2.csv', 'tmp_data\\pp-1996-part1.csv', 'tmp_data\\pp-2011-part1.csv', 'tmp_data\\pp-2017-part1.c

In [ ]:
# fynesse.access.create_pp_table(pp_database_conn)

In [ ]:
# 15m
fynesse.access.upload_files_to_table(pp_database_conn, downloaded_pathnames, 'pp_data')

In [ ]:
# 8m 30s
fynesse.access.setup_pp_table(pp_database_conn)

In [ ]:
fynesse.access.select_count(pp_database_conn, 'pp_data')

In [ ]:
# fynesse.access.create_postcode_table(pp_database_conn)

In [8]:
# 10s
postcode_filename = fynesse.access.download_postcode_data()

tmp_data


In [ ]:
# 1m
fynesse.access.upload_files_to_table(pp_database_conn, [postcode_filename], 'postcode_data')

In [ ]:
# 1m
fynesse.access.setup_postcode_table(pp_database_conn)

### Task D

This table should contain the house price paid and the latitude and longitude of the house. We could create a new data frame that contains all this information. However, the computation of that data frame would take some time because of the size of the two existing tables in the join. Whether this is a good idea or not in a live system will depend on how often these predictions are required. If it's very often, it would be better to store the join as a new table in the database, because the one-off cost for that join is amortised across all the future predictions. If only a few predictions are required (like in our lab class) then doing that join on the fly might be better.

- Option A: Think about which columns from each table will be useful to you in making predictions, then write code to efficiently select this information and create a data frame from the two tables for a set of properties. "Join on the fly". This option looks easier but the disadvantage is the extra latency it adds as joins are performed every time we need to answer data questions. These latencies are usually not accepted in production environments.

- Option B: Alternatively, you can create a database table called `property_prices` to store the join of the tables you created in the previous tasks. The advantage of this approach is that you will get faster responses because the join is performed once. The disadvantage is that populating the new table can take a long time because you would join two big tables. You need to be more creative with this option. Remember that you can divide your dataset by different criteria (e.g., by year) and that loading data from files is much faster than `INSERT INTO` instructions. Populating the table took from 4 to 6 minutes per year in our tests on a Dell Laptop Intel Core i5 with 16GB of RAM and using the Eduroam network at the Computer Lab. Populating the table by year also gives you control over the upload process. You could write your code in a way you can stop and restart the upload process. The new table could have a schema like the one below:

  ```
  USE `property_prices`;
  --
  -- Table structure for table `prices_coordinates_data`
  --
  DROP TABLE IF EXISTS `prices_coordinates_data`;
  CREATE TABLE IF NOT EXISTS `prices_coordinates_data` (
    `price` int(10) unsigned NOT NULL,
    `date_of_transfer` date NOT NULL,
    `postcode` varchar(8) COLLATE utf8_bin NOT NULL,
    `property_type` varchar(1) COLLATE utf8_bin NOT NULL,
    `new_build_flag` varchar(1) COLLATE utf8_bin NOT NULL,
    `tenure_type` varchar(1) COLLATE utf8_bin NOT NULL,
    `locality` tinytext COLLATE utf8_bin NOT NULL,
    `town_city` tinytext COLLATE utf8_bin NOT NULL,
    `district` tinytext COLLATE utf8_bin NOT NULL,
    `county` tinytext COLLATE utf8_bin NOT NULL,
    `country` enum('England', 'Wales', 'Scotland', 'Northern Ireland', 'Channel Islands', 'Isle of Man') NOT NULL,
    `latitude` decimal(11,8) NOT NULL,
    `longitude` decimal(10,8) NOT NULL,
    `db_id` bigint(20) unsigned NOT NULL
  ) DEFAULT CHARSET=utf8 COLLATE=utf8_bin AUTO_INCREMENT=1 ;

  ```

In both cases you will need to perform a join between `pp_data` and `postcode_data` tables. Joining large tables without the right indexes in place will take a long time. You should think and set the right index for an efficient join. Indexing the `pp_data` table should take less than 5 minutes, while it takes less than one minute to index the `postcode_data` table.

Note that there is no preference for either approach in the mark scheme.

You should use the joined data in your prediction model in Question 3. Exploit the nature of the task to use prices for a particular region in a given period. This means we can select the relevant rows from the database according to that region and period.

***After you have populated your database tables and created the functions to access the data you need for Question 3, you will not need to redo any of the previous steps. If at some point you find the AWS database is not responding or taking longer than expected to perform operations, you can have a look at the process list and kill the one are causing problems. If killing the processes does not work, you should reboot the database in the AWS console. Be careful with other database instances if you need to reboot your database. Also, be careful not to delete the database instead of rebooting it. If you delete the database, it is likely you will need to redo all Question 1.***

In [ ]:
# Write the code you used to join the tables, or the code you used to join on the fly.
fynesse.access.create_prices_coordinates_table(pp_database_conn)

In [52]:
joined_table_pathnames = fynesse.access.join_all_tables(downloaded_pathnames, postcode_filename, overwrite=False)

In [53]:
# 10m
fynesse.access.upload_files_to_table(pp_database_conn, joined_table_pathnames, 'prices_coordinates_data', ignore_first_row=True)

In [54]:
# 9m
fynesse.access.setup_prices_coordinates_table(pp_database_conn)

## Question 2. Accessing OpenStreetMap and Assessing the Available Features

In question 3 you will be given the task of constructing a prediction system for property price levels at a given location. We expect that knowledge of the local region around the property should be helpful in making those price predictions. To evaluate this we will now look at [OpenStreetMap](https://www.openstreetmap.org) as a data source.

The tasks below will guide you in accessing and assessing the OpenStreetMap data. The code you write will eventually be assimilated in your python module, but documentation of what you've included and why should remain in the notebook below.

Accessing OpenStreetMap through its API can be done using the python library `osmx`. Using what you have learned about the `osmx` interface in the lectures, write general code for downloading points of interest and other relevant information that you believe may be useful for predicting house prices. Remembering the perspectives we've taken on *data science as debugging*, the remarks we've made when discussing *the data crisis* of the importance of reusability in data analysis, and the techniques we've explored in the labsessions for visualising features and exploring their correlation use the notebook to document your assessment of the OpenStreetMap data as a potential source of data.

The knowledge you need to do a first pass through this question will have been taught by end of lab session three (16th November 2021). You will likely want to review your answer as part of *refactoring* your code and analysis pipeline shortly before hand in.

You should write reusable code that allows you to explore the characteristics of different points of interest. Looking ahead to question 3 you'll want to incorporate these points of interest in your prediction code.

*5 marks*


In [269]:
# Use this cell and cells below for summarising your analysis and documenting your decision making.

# PCA of training dataset


In [270]:
from datetime import datetime

C:\Users\86189\AppData\Local\Temp\ipykernel_12324\1078233914.py:83: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)


## Question 3. Addressing a Property Price Prediction Question

For your final tick, we will be asking you to make house price predictions for a given location, date and property type in the UK. You will provide a function that takes input a latitude and longitude as well as the `property_type` (either type" of property (either `F` - flat, `S` - semidetached, `D` - detached, `T` - terraced or `O` other). Create this function in the `address.py` file, for example in the form,

```
def predict_price(latitude, longitude, date, property_type):
    """Price prediction for UK housing."""
    pass
```

We suggest that you use the following approach when building your prediction.

1. Select a bounding box around the housing location in latitude and longitude.
2. Select a data range around the prediction date.
3. Use the data ecosystem you have build above to build a training set from the relevant time period and location in the UK. Include appropriate features from OSM to improve the prediction.
4. Train a linear model on the data set you have created.
5. Validate the quality of the model.
6. Provide a prediction of the price from the model, warning appropriately if your validation indicates the quality of the model is poor.

Please note that the quality of predictions is not the main focus of the assignment - we expect to see models that output reasonable predictions and have positive R^2's, but you should not spend too much time on increasing the model's accuracy.

The knowledge you need to do a first pass through this question will have been taught by end of lab session four (7th November 2023). You will likely want to review your answer as part of *refactoring* your code shortly before hand in.



In [314]:
fynesse.assess.predict_price(52.206767, 0.119229, datetime(2021, 1, 1), 'S')

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.373
Model:                            OLS   Adj. R-squared:                  0.318
Method:                 Least Squares   F-statistic:                     6.796
Date:                Tue, 21 Nov 2023   Prob (F-statistic):           8.79e-11
Time:                        22:01:13   Log-Likelihood:                -2926.5
No. Observations:                 175   AIC:                             5883.
Df Residuals:                     160   BIC:                             5930.
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.308e+09    1.7e+09      0.770      0.4

9442202.030616283

## Large Language Models

If you used LLMs to generate or fix code in this assignment (recommended), briefly summarise the process and prompts you used. What do you think of the integration of LLMs in the data science pipeline?

```GIVE YOUR WRITTEN ANSWER HERE```

### FAQs

- Why is my connection to the AWS server intermittent?

If you are using Google Colab, online notebooks may throttle your code or time you out. Local Python code is recommended for tasks for large data management in the database.

- Why do SQL queries take a long time?

Some queries legitimately take a long time, even when done right. We suggest indexing your tables to speed up queries over your database. You can index tables by different columns depending on the queries you want to perform. For example, indexing the tables by `postcode` could speed up the join in Task D. More information regarding indexing in MariaDB is available [here](https://mariadb.com/kb/en/getting-started-with-indexes/).

You may also want to consider creating single or multi-column indices on coordinates, or any other properties you use to select data, if that step is taking a long time.

If your new queries seem stuck, try running `SHOW FULL PROCESSLIST`, and `KILL` any stuck processes.

- Why are table populating processes taking so long?

Again populating the database can take long. However, be careful if you are indexing the tables. You should populate data before indexing. Insert operations are impacted by indexes as they are updated with each new row inserted into the table.

- Some other questions are answered in [this reddit forum](https://www.reddit.com/r/CST_ADS/) or [this doc](https://docs.google.com/document/d/1GfDROyUW8HVs2eyxmJzKrYGRdVyUiVXzPcDfwOO8wX0/edit?usp=sharing). Feel free to also ask about anything that comes up.